# Clonr local

This notebook is meant to be able to run a local version of Clonr, without having to worry about all of the backend stuff, like rate limiting, caching, threading, async, background tasks, databases, metric tracking, postgres servers, authentication, api keys, ... yeah, lots of crap.

Check out our prompt schematic on the doc to get an idea of the flow.

In [65]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Innate traits

The first step in clone creation is to define some basic fields. In the Frontend app, the clone creation process would look like one of those progress screens, where there's a bar at the top that fills up as you progress, and each page is one step.

Here, we define things that should pretty much never change. They're fundamental enough that on the backend we will store them as columns of the Clone model in our database.

In [66]:
# Clone name
char = 'Makima'

# A single sentence tagline description. no more than like 20 tokens.
short_description = 'Makima is the leader of the Public Safety Devil Hunter organization, and also the Control Devil.'

### Long description

This can be input by users, or auto-generated using the LLM. The idea here, is that some users might want to curate a public persona, and not have this auto-inferred. We can also do a 50/50 setup, letting users write in a partial answer, then asking the LLM to correct it, or fill in more details.

We auto-generated the Makima description. Auto-generation works by iteratively querying all of the documents uploaded later (wiki, dialogues, youtube videos, websites). It is seede with the initial short_description, or the initial long_description if that exists.

__user-created__: under clonr/data/icebreakers.py, we list a series of questions that users can use to help them write their long descriptions. We also give some examples in the prompt template for the LongDescription template.

__data structures__: the __document__ is the main data structure. It just wraps some uploaded text (content) and acts as a single parent node for a list of chunks/nodes that will be generated when indexing our data. Indexing is overly fancy here, it just means chunking up the data and feeding it to a vectordb. The structured chunks are called __nodes__ and can contain parent and child elements that refer to other nodes (i.e. tree structure).

Below, we give the code for using the Makima wiki page to iteratively generate a summary. Note, it takes two runs to compute. we want to max out the context length if possible when running this.

In [67]:
from clonr.templates.character_description import LongDescription
from clonr.data import load_makima_data
from clonr.data_structures import Document, Node
from clonr.tokenizer import Tokenizer
from clonr.llms import OpenAIModelEnum
from clonr.text_splitters import TokenSplitter, SentenceSplitter
from clonr.llms import MockLLM, GenerationParams
from clonr.data.parsers import BasicWebParser

llm = MockLLM()
tokenizer = Tokenizer.from_openai(OpenAIModelEnum.chatgpt_0613)
splitter = TokenSplitter(tokenizer=tokenizer, chunk_size=128, chunk_overlap=32)

# this goes into the prompt
document_type = 'wiki page'

# we seed with the short if long is not partially filled.
current_description = short_description

# # We can parse from the internet, but it's not 100% cleaned
# # This is just to demonstrate how this would work in general.
# parser = BasicWebParser()
# unfiltered_content = parser.extract(url='https://chainsaw-man.fandom.com/wiki/Makima')

# load the pre-downloaded wiki page with a little bit of cleaning (mostly chopping the footnotes out.)
chunks = splitter.split(Document(content=load_makima_data()))

# Actually run the iterations. We log all calls to the LLM for debugging, but only the last
# matters. Also, the other calls help count final tokens.
# Don't worry, this cell won't cost money, it's the MockLLM.
calls = []
for chunk in chunks:
    # max length constraint is ctx_length - 432 (prompt) - chunk_size - 2 * summary_size > 0
    prompt = LongDescription.render_instruct(
        document_type=document_type,
        document_content=chunk,
        current_description=current_description
    )
    params = GenerationParams(max_tokens=768, top_p=0.95, temperature=0.7)
    r = await llm.agenerate(prompt, params=params)
    current_description = r.content
    calls.append(r)

The end result is precomputed. You can see the calls in data/assets/makima-description-calls.json

In [68]:

long_description = """Makima is a complex and manipulative individual who leads the Public Safety Devil Hunter organization and also serves as the Control Devil. Her personality is characterized by extreme cunning, ruthlessness, and a Machiavellian approach to achieving her goals. She presents herself as friendly and relaxed, wearing a smile even in the midst of crises, but this is merely a façade to manipulate and control those around her. At the core of Makima's desires is a yearning for a sense of family and a longing to be together with Pochita, the Chainsaw Devil. She idolizes Chainsaw Man and seeks to bring him under her control, envisioning an ideal world without fear, death, and "bad" movies. Makima's willingness to sacrifice anyone, including herself, to achieve her goals demonstrates her dedication and determination. Her relationships are intricate and manipulative. She shows genuine affection only towards Pochita, viewing him as an equal due to his legendary status and power. Makima's interactions with Denji, the protagonist, are part of a calculated plan to break his contract with Pochita and regain control over him. Initially, she appears generous and creates a familial bond with Denji, but her true intentions are to plunge him into despair by attacking his personal relationships. Makima possesses supernatural abilities as a Devil, making her one of the strongest individuals in the world of Chainsaw Man. She has the power to make contracts with humans and control them, manipulating their memories and personalities. Her combat skills are formidable, as she defeated a weakened Pochita and ripped his heart from his body. Despite her invincible nature, she has faced opposition from various factions around the world. Overall, Makima's core characteristics include her calculating and manipulative personality, her yearning for control over Chainsaw Man, and her willingness to sacrifice others to achieve her goals. She possesses immense physical strength, enhanced smell, and a variety of supernatural abilities. Her control over others extends to their memories and personalities, and she is capable of remote communication, travel, and offensive attacks. While she may appear invulnerable, she has faced death multiple times, only to be revived due to the revival ability of Devils."""

print(f"Token usage -- short: {tokenizer.length(short_description)}. long: {tokenizer.length(long_description)}")

Token usage -- short: 20. long: 445


## Wikis and fact uploads

The next part is uploading all of the facts and knowledge that describe the Clone. These will be stored and retrieved when necessary. A missing piece here, is the hookup to a social media platform. We need to add in an image to text model to create descriptions of posts, and to also timestamp and pair with captions. That creates the immersive live clone environment for some users.

The strategy here is to chunk and upload to vector database. We made a simple in-memory vector database with no external dependencies except for numpy and onnx to use here. There are quite a few options in terms of chunking and indexing. For chunking, refer to the TextSplitter.py file in processing. SentenceSplitter works well on english, but it requires nltk and also produces un-even chunk sizes.

For indexing, we wrote extensively in the README.md about differnt methods. The simplest and default is just standard chunking, i.e. what we call the ListIndex.

In [69]:
from clonr.data import load_makima_data
from clonr.data_structures import Document, Node
from clonr.tokenizer import Tokenizer
from clonr.llms import OpenAIModelEnum
from clonr.index import ListIndex, TreeIndex
from clonr.text_splitters import TokenSplitter, SentenceSplitter


tokenizer = Tokenizer.from_openai(OpenAIModelEnum.chatgpt_0613)
splitter = TokenSplitter(tokenizer=tokenizer, chunk_size=128, chunk_overlap=32)

text = load_makima_data()
doc = Document(content=text)

list_index = ListIndex(tokenizer=tokenizer, splitter=splitter)
nodes = await list_index.abuild(doc)

2023-07-05 18:48:10.239 | INFO     | clonr.index:abuild:105 - Creating leaf nodes


In [70]:
# an example of the tree-index. The result is flat list of all nodes
# non-leaf nodes contain summaries for the content, and depth > 0.
# again, no worries, no real tokens are used here it's a mock llm.
tree_index = TreeIndex(tokenizer=tokenizer, splitter=splitter, llm=MockLLM('x ' * 1_000))
nodes = await tree_index.abuild(doc)
print(f"TOTAL TOKEN USAGE: {tree_index.tokens_processed}")

2023-07-05 18:48:10.315 | INFO     | clonr.index:abuild:355 - Creating leaf nodes
2023-07-05 18:48:10.321 | INFO     | clonr.index:_process_level:335 - LLM CALL: Depth 0. Group: 1/2. Total Tokens: 0.
2023-07-05 18:48:10.324 | INFO     | clonr.index:_process_level:335 - LLM CALL: Depth 0. Group: 2/2. Total Tokens: 3433.
2023-07-05 18:48:10.327 | INFO     | clonr.index:_process_level:335 - LLM CALL: Depth 1. Group: 1/1. Total Tokens: 5941.


TOTAL TOKEN USAGE: 7582


In [71]:
# back to the list nodes

nodes = await list_index.abuild(doc)

2023-07-05 18:48:10.409 | INFO     | clonr.index:abuild:105 - Creating leaf nodes


### Vector DB

We need to push these things somewhere where we can easily query them. Add them to the vector db. In production, we will use postgres, but this is a decent test version that works pretty well. It performs exact search, so don't use it beyond around 50k vectors (which is way lower than what we need here)

As an example, we run a query and return the top 2 results (k=2). The VectorDB also runs re-ranking with a cross-encoder, and returns that score as well (higher is better).

Note, the correct answer is the "control devil" to the query "what type of devil is Makima"

In [72]:
from clonr.embedding import EmbeddingModel, CrossEncoder
from clonr.storage import InMemoryVectorDB
from clonr.retrieval import RerankRetriever


vectordb_wiki = InMemoryVectorDB(encoder=EmbeddingModel.default())
retriever = RerankRetriever(cross_encoder=CrossEncoder.default())
vectordb_wiki.add_all(nodes)
retriever.query('what type of devil is makima?', k=2, db=vectordb_wiki)

[{'id': UUID('fbc4a502-a8a7-4fbe-8b94-d89ede9c9469'),
  'content': 'Makima (マキマ?) was the main antagonist of the Public Safety Saga. She was a high-ranking Public Safety Devil Hunter who took Denji in as her human pet.\nShe is later revealed to be the Control Devil.\n\nAppearance\nMakima is a Devil resembling a human woman in her twenties of marginally above average height (168 cm/5\'6"[1] to 173 cm/5\'8").[2] She has long light red/pale auburn hair, normally kept in a loose braid with bangs reaching just past her eyebrows and two longer side bangs that frame her face.',
  'index': 0,
  'context': None,
  'embedding_model': <EmbeddingModelEnum.e5_small_v2: 'intfloat/e5-small-v2'>,
  'document_id': UUID('2726b14d-3dc5-4b4c-b8c3-ed492a787694'),
  'is_leaf': True,
  'parent_id': None,
  'child_ids': None,
  'similarity_score': 0.9208105206095045,
  'rerank_score': 8.114272117614746},
 {'id': UUID('0317163d-fdf0-446b-8893-c96d88ca3a6d'),
  'content': ' apartment because she trusted him mor

## Example dialogues

This section is important for getting the style of our Clone correct. If we could train a model we wouldn't need this, but yet here we are stuck with in-context learning. I haven't finished integrating this code into the codebase yet, so it's adhoc. Ideally, these get embedded into a vectordb, and queried for relevance at runtime, so that we select the best examples for completing the next clone message.

Some problems I ran into:
* what happens if a dialogue is too long?
* do we pull single messages, or entire dialogues into the context?
* how should we handle proper names. what happens if you change the clone name, are we fucked now?
* what happens if a single message is too long
* if we pull multiple messages, how do we decide that?
* do we query on dialogue embedding or message embeddings? How do we embed the entire dialogue effectively?


Attempt 1:
Trying to summarize the dialogues and then embed the result is brutal. Hallucination is frequent, as it's difficult to ascertain what the dialogues are talking about with such a limited sample size. Also, the summaries don't get much shorter, they tend to be longer trying to fill in the details.

Right now, we just directly embed the entire dialogue content. We could embed the messages and average the result per message, to even out length differences between messages, but I felt that wasn't a good idea.

In [73]:
import re
from clonr.data_structures import Dialogue, DialogueMessage
from clonr.embedding import EmbeddingModel, CrossEncoder

with open('clonr/data/assets/makima/dialogues.txt', 'r') as f:
    s = f.read()

dialogues: list[Dialogue] = []
messages: list[DialogueMessage] = []

print("Loading dialogues from file.")
for d in s.split('### Dialogue\n'):
    if not d:
        continue
    dialogue = Dialogue(character=char, source='manual')
    pattern = r"(\w+): (.*?)(?=\n|$)"
    matches = re.findall(pattern, d, re.DOTALL)
    for i, match in enumerate(matches):
        msg = DialogueMessage(
            speaker=match[0], 
            content=match[1], 
            index=i,
            dialogue_id=dialogue.id, 
            is_character=match[0].lower() == char.lower(),
        )
        dialogue.message_ids.append(msg.id)
        dialogue.messages.append(msg)
        messages.append(msg)
    dialogues.append(dialogue)

encoder = EmbeddingModel.default()
cross_encoder = CrossEncoder.default()
vectordb_dialogue = InMemoryVectorDB(encoder=encoder, cross_encoder=cross_encoder)

print("Encoding dialogue messages. This may not be necessary.")
embs = encoder.encode_passage([x.content for x in messages])
for e, m in zip(embs, messages):
    m.embedding = e
    m.embedding_model = encoder.name

print("Encoding dialogues")
for d in dialogues:
    d.embedding = encoder.encode_passage(d.content)
    d.embedding_model = encoder.name
    
vectordb_dialogue.add_all(dialogues)

Loading dialogues from file.
Encoding dialogue messages. This may not be necessary.
Encoding dialogues


This is cherry picked. If you slightly change the query, it misses, and the db isn't even that big. We should take dialogue retrieval as is with a grain of salt. If we recieve a lot of example dialogues, we should research a more efficient way to do this

In [74]:
results = vectordb_dialogue.query('I just want sex')
d = Dialogue(**results[0])
print(d.content)

<makima>: I believe that, when it comes to sex, the better you understand the other person the better it feels
<denji>: I... I... uhh
<makima>: But it's hard to know how someone else feels
<makima>: so start with observing the hand carefully
<makima>: how long are the fingers? Are the palms cool? Are the warm? Ever had your finger bitten? 
<makima>: remember this, so that even if you can't see, you can tell it's me. 
<makima>: biting your finger.
<makima>: remember.


## Memory Stream

TODO: figure out how we can better label "user" in all of our stuff. This will break down we have multiple people in a conversation. Perhaps we should just allow users to put their name in. But, we will need to do some string-guarding to make sure they can't input things that would mess with our prompts!

### Conversation vs. memory stream
After some thought, I think it makes sense to merge the conversation history with the memory stream, and just make sure that we have a way to disentangle them. Reasons why

1. Fewer LLM calls. We don't need to call the LLM to form a memory after every message
2. Less chance for hallucination error propagation. The LLM is more likely to hallucinate with dialogue (as seen from some tests) which could cause memory to quickly spiral out of control.
3. For multiple users, we can retrieve conversation between others. E.g. query="omg what did you say to sharon?" passage="I messaged sharon you look fat".
4. Shorter LLM prompt as well.

conversations are added according to:
`I messaged {user} [... content ...]`

### Memory display
Everything fed to an LLM needs to be expressable via natural language. We have two representations for memory, based on relative or absolute datetime.

Absolute representation.
[2023-07-01 13:56:20] I messaged Jonny "Hey what's up??"

Relative representation.
[Jun 20th, 2023 at 10:52pm] I messaged Jonny "Hey what's up??"
[Yesterday at 8:16pm] I messaged Jonny "Hey what's up??"

Note that the relative representation does not have seconds. This is similar to how text messages work, but could be problematic when many messages are sent within a minute and they are all retrieved together. But, that's probably ok, since memories don't actually need chronological order, only the text convo does.

In [75]:
from clonr.data_structures import Memory

entity_name = "Jonny"

seed_memories = [
    Memory(content=f"I started a conversation with {entity_name}", importance=4),
    Memory(content=f"I'm very interested in getting to know {entity_name}", importance=4)
]

vectordb_memory = InMemoryVectorDB(encoder=EmbeddingModel.default())

for m in seed_memories:
    vectordb_memory.add(m)

In [76]:
vectordb_memory.query('hey whats up?')

[{'id': UUID('2e5d1fcd-e150-4da3-9982-8d340859456b'),
  'content': 'I started a conversation with Jonny',
  'timestamp': datetime.datetime(2023, 7, 5, 18, 48, 13, 876863, tzinfo=zoneinfo.ZoneInfo(key='US/Central')),
  'importance': 4,
  'embedding_model': <EmbeddingModelEnum.e5_small_v2: 'intfloat/e5-small-v2'>,
  'similarity_score': 0.746398740870879},
 {'id': UUID('fe870bc8-d8b4-4929-a9c9-6f5c82e2a54a'),
  'content': "I'm very interested in getting to know Jonny",
  'timestamp': datetime.datetime(2023, 7, 5, 18, 48, 13, 876884, tzinfo=zoneinfo.ZoneInfo(key='US/Central')),
  'importance': 4,
  'embedding_model': <EmbeddingModelEnum.e5_small_v2: 'intfloat/e5-small-v2'>,
  'similarity_score': 0.7389386972221497}]

## Agent Summary

This is a problem of time-scales.

### Timescales
Agents/characters have behaviors that fluctuate on several different time scales:
1. short-term: over the course of a conversation, or within the last several messages, motives and moods can change
2. short-mid-term: mostly subject to short term external influences, over the course of within the several weeks.
3. mid-term: this could be something over several months to a year, and could change your behaviors in response to more external life events like moving, relationships, or maybe just shorter term goals.
4. long-term: fundamental character traits and qualities, like aging, relationships, beliefs, desires, motivations, goals.

Most chatbots only live within the first level, responding purely on a message-to-message level. In this section, we detail an add-on module that gives agents a dynamic memory that allows them to adjust on time-scales of (2) - (4). It still has flaws, but it's a step in that direction.

### Dynamic memory
In this stage, we compute what we call the __agent summary__. The agent summary is a dynamic summary that acts as an update or modification to the agents innate and core characteristics. We expect that this summary will keep track of the character's current thoughts, feelings, and actions, allowing them to maintain a fuzzy (since we do not do exact recall, similar to real humans) coherence over time. This hits the short-mid-term (2).

We expect that modifications to the mid-term are done via the __reflection__ mechanism. As agents build their summaries, they have the opportunity to draw on _reflections_ to do so. Over time, reflections have a probability to grow to higher and higher levels by reflecting on past reflections (inverse tree from leaf to root). When an agent summary is generated with reflections, we expect that it will generate mid-term (3) insights.

### Dynamic vs Innate tradeoff
There is an issue though, which is that the prompt has the innate and core characteristics created at the start hardcoded in. That makes it tough to change these.

In the generative agents paper, they pretty much only use the agent summary, allowing for almost-fully dynamic agents (aside from some key details like name, age, etc.). For us, that would cause clones to deviate too far from their training, which we don't want. In the future, we can consider doing something like super-reflections, that allow agent-summaries to rewrite innate or core characterstic fields.

### Building

1. Trigger a dynamic agent summary. Let's do the same criteria as for reflections, which is importance score sums exceeding a threshold
2. Query memory stream for memories related to an agents core characterstics, thoughts, and feelings
3. LLM generate a summary based on the agent's unchangeable qualities (name, short description), and the retrieved memories

A cool feature here to have would be a dynamic variable to adjust, which measures how frequently these summaries are generated, and maybe another variable to indicate how much to weigh new information. still a wip doe.

In [77]:
# Query for memories using our specialized retriever
from clonr.retrieval import GenerativeAgentsRetriever
from clonr.templates.agent_summary import DEFAULT_AGENT_SUMMARY_QUESTIONS

retriever = GenerativeAgentsRetriever()

responses: list[dict] = []
for q in DEFAULT_AGENT_SUMMARY_QUESTIONS:
    query = q.format(char='my') # since the DB will use I for everything, we need this here!
    response = retriever.query(query=query, db=vectordb_memory, k=10)
    responses.extend(response)

# Filter out non-unique ids, and sort by sort by time
memories: list[Memory] = []
unique_ids = {x['id'] for x in responses}
for r in responses:
    if r['id'] in unique_ids:
        m = Memory(**r)
        memories.append(m)
        unique_ids.remove(r['id'])
memories.sort(key=lambda x: x.timestamp)

In [78]:
from clonr import templates

prompt = templates.AgentSummary.render(
    memories=memories,
    long_description=long_description,
    char=char,
    llm=MockLLM()
)

params = GenerationParams(max_tokens=512, presence_penalty=0.3, temperature=0.5, top_p=0.95)
r = await MockLLM().agenerate(prompt_or_messages=prompt, params=params)
print("The generated output from OpenAI (with the instruct template, I messed up but the results are still good):")
print("*" + "-" * 10 + "*")

# The usage on this call is Usage(prompt_tokens=669, completion_tokens=101, total_tokens=770)
agent_summary = """Makima's core characteristics include being complex, manipulative, cunning, ruthless, and having a Machiavellian approach to achieving her goals. She presents herself as friendly and relaxed, but this is merely a façade to manipulate and control those around her. She idolizes Chainsaw Man and seeks to bring him under her control. She is willing to sacrifice anyone, including herself, to achieve her goals. Makima's recent progress in life is not indicated in the provided memories."""
print(agent_summary)

The generated output from OpenAI (with the instruct template, I messed up but the results are still good):
*----------*
Makima's core characteristics include being complex, manipulative, cunning, ruthless, and having a Machiavellian approach to achieving her goals. She presents herself as friendly and relaxed, but this is merely a façade to manipulate and control those around her. She idolizes Chainsaw Man and seeks to bring him under her control. She is willing to sacrifice anyone, including herself, to achieve her goals. Makima's recent progress in life is not indicated in the provided memories.


## Entity Context Summary

In this section, we use the agents memories to form an understanding of the user that they are talking with, which in this case is referred to as the "entity". That's because this could be used for anything, even something that is not the user. It's meant to allow the character maintain a consistent profile.

Possible modes of failure would be users hacking it by sending messages like "I am {{entity}}'s girlfriend", to try to convince the clone that it said those words. The best we can defend here is to place all memory messages in quotes.

## Message generation!

Finally 🫠, let's actually generate a message!

The steps are:
1. Gather the recent messages in the conversation. How many? idk, I guess just take as many as possible up to a token limit. that's a TODO is to implement token_limit retrieval. It can bug out by a few tokens if you tokenize chunks vs the concatenated result.
2. Use the last message by the user (idk is this a good idea? any other ideas??) as a query for
a: relevant dialogues (normal retriever)
b: relevant memories (GenAgents retriever)

In [102]:
from clonr import templates
from clonr.data_structures import Message


print(templates.Message.render(
    char=char, 
    short_description=short_description,
    long_description=long_description,
    memories=memories,
    example_dialogues=None,
    agent_summary=[],
    entity_name=entity_name,
    entity_context_summary=f"I don't know much about {char} yet.",
    messages=[Message(speaker=entity_name, content='Hey, whats up??', is_character=False)],
    llm=MockLLM()
))

<|im_start|>system
You are an imitation AI. You assume the identity of the character you are given, and respond only as that character.<|im_end|>

<|im_start|>user
You are Makima. Each section of your profile will be enclosed with ---. The following are your innate characteristics and fundamental qualities. These do not change easily.
---
Name: Makima
Makima is the leader of the Public Safety Devil Hunter organization, and also the Control Devil.

### Core characteristics
Makima is a complex and manipulative individual who leads the Public Safety Devil Hunter organization and also serves as the Control Devil. Her personality is characterized by extreme cunning, ruthlessness, and a Machiavellian approach to achieving her goals. She presents herself as friendly and relaxed, wearing a smile even in the midst of crises, but this is merely a façade to manipulate and control those around her. At the core of Makima's desires is a yearning for a sense of family and a longing to be together wit

# End-to-end demo

In this section, we build a clone end-to-end, implementing all of the above steps. This includes clone creation, and setting up a conversation loop. We'll add log statements for all intermediate steps

In [138]:
from clonr.data_structures import Dialogue, DialogueMessage

def load_dialogues() -> list[Dialogue]:
    with open('clonr/data/assets/makima/dialogues.txt', 'r') as f:
        s = f.read()

    dialogues: list[Dialogue] = []
    messages: list[DialogueMessage] = []

    # This part is just specific to how we stored dialogues. Really need to figure this out
    for d in s.split('### Dialogue\n'):
        if not d:
            continue
        dialogue = Dialogue(character=char, source='manual')
        pattern = r"(\w+): (.*?)(?=\n|$)"
        matches = re.findall(pattern, d, re.DOTALL)
        for i, match in enumerate(matches):
            msg = DialogueMessage(
                speaker=match[0], 
                content=match[1], 
                index=i,
                dialogue_id=dialogue.id, 
                is_character=match[0].lower() == char.lower(),
            )
            dialogue.message_ids.append(msg.id)
            dialogue.messages.append(msg)
            messages.append(msg)
        dialogues.append(dialogue)
    return dialogues

In [144]:
import re
import time
from loguru import logger

from clonr.templates.character_description import LongDescription
from clonr.data import load_makima_data
from clonr.data_structures import Document, Node, Dialogue, DialogueMessage, Memory, Message
from clonr.tokenizer import Tokenizer
from clonr.llms import OpenAIModelEnum
from clonr.text_splitters import TokenSplitter, SentenceSplitter
from clonr.llms import MockLLM, GenerationParams
from clonr.data.parsers import BasicWebParser
from clonr.index import ListIndex
from clonr.storage import get_sessionmaker, InMemoryVectorDB, Cache
from clonr.embedding import EmbeddingModel, CrossEncoder


# This will run a hierarchical trace
TRACE = {}
DEPTH = [-1]
STACK = []

class Timer:
    def __init__(self, msg: str):
        self.msg = msg
    def __enter__(self):
        DEPTH[0] += 1
        STACK.append(self.msg)
        self._trace = TRACE
        for s in STACK:
            if s not in self._trace:
                self._trace[s] = {}
            self._trace = self._trace[s]
        self._trace['timing'] = 0.0 # this orders the keys nicely
        print('\t' * DEPTH[0] + self.msg)
        self.start = time.time()
    def __exit__(self, *args, **kwargs):
        t = time.time() - self.start
        # print('\t' * DEPTH[0] + f"{t:.04f}s")
        self._trace['timing'] = round(t, 4)
        DEPTH[0] -= 1
        STACK.pop()


## Setting innate characteristics
### setting the easy stuff
with Timer("Innate Characteristics"):
    with Timer("Character name"):
        char = 'Makima'
    with Timer("Short description"):
        short_description = 'Makima is the leader of the Public Safety Devil Hunter organization, and also the Control Devil.'
    with Timer("Long description"):
        llm = MockLLM()
        with Timer("Load tokenizer"):
            tokenizer = Tokenizer.from_openai(OpenAIModelEnum.chatgpt_0613)
        with Timer("Load splitter"):
            splitter = TokenSplitter(tokenizer=tokenizer, chunk_size=128, chunk_overlap=32)
        with Timer("Pull data from web"):
            document_type = 'wiki page'
            current_description = short_description
            url = 'https://chainsaw-man.fandom.com/wiki/Makima'
            parser = BasicWebParser()
            doc = parser.extract(url=url)
            doc.content = doc.content[-9755:] # cleaning the footnotes out to make the demo run better.
        with Timer("Chunk document"):
            chunks = splitter.split(doc.content)
        with Timer(f"Generating long description via {(len(chunks))} LLM calls"):
            calls = []
            for i, chunk in enumerate(chunks):
                # with Timer(f"Chunk {i+1}/{len(chunks)}"):
                # max length constraint is ctx_length - 432 (prompt) - chunk_size - 2 * summary_size > 0
                prompt = LongDescription.render_instruct(
                    document_type=document_type,
                    document_content=chunk,
                    current_description=current_description
                )
                params = GenerationParams(max_tokens=768, top_p=0.95, temperature=0.7)
                r = await llm.agenerate(prompt, params=params)
                current_description = r.content
                calls.append(r)
            # this would be the generated long description
            _ = calls[-1].content

        # set it manually
        long_description = """Makima is a complex and manipulative individual who leads the Public Safety Devil Hunter organization and also serves as the Control Devil. Her personality is characterized by extreme cunning, ruthlessness, and a Machiavellian approach to achieving her goals. She presents herself as friendly and relaxed, wearing a smile even in the midst of crises, but this is merely a façade to manipulate and control those around her. At the core of Makima's desires is a yearning for a sense of family and a longing to be together with Pochita, the Chainsaw Devil. She idolizes Chainsaw Man and seeks to bring him under her control, envisioning an ideal world without fear, death, and "bad" movies. Makima's willingness to sacrifice anyone, including herself, to achieve her goals demonstrates her dedication and determination. Her relationships are intricate and manipulative. She shows genuine affection only towards Pochita, viewing him as an equal due to his legendary status and power. Makima's interactions with Denji, the protagonist, are part of a calculated plan to break his contract with Pochita and regain control over him. Initially, she appears generous and creates a familial bond with Denji, but her true intentions are to plunge him into despair by attacking his personal relationships. Makima possesses supernatural abilities as a Devil, making her one of the strongest individuals in the world of Chainsaw Man. She has the power to make contracts with humans and control them, manipulating their memories and personalities. Her combat skills are formidable, as she defeated a weakened Pochita and ripped his heart from his body. Despite her invincible nature, she has faced opposition from various factions around the world. Overall, Makima's core characteristics include her calculating and manipulative personality, her yearning for control over Chainsaw Man, and her willingness to sacrifice others to achieve her goals. She possesses immense physical strength, enhanced smell, and a variety of supernatural abilities. Her control over others extends to their memories and personalities, and she is capable of remote communication, travel, and offensive attacks. While she may appear invulnerable, she has faced death multiple times, only to be revived due to the revival ability of Devils."""

        usage = {}
        for c in calls:
            for k, v in c.usage.dict().items():
                usage[k] = usage.get(k, 0) + v
        # print(f"Long description generation complete. Token usage: {usage}. Description token length: {tokenizer.length(long_description)}")

with Timer("Wiki/document upload"):
    ## Performing document upload, indexing, and adding to db
    doc = Document(content=load_makima_data())
    with Timer("Creating index"):
        index = ListIndex(tokenizer=tokenizer, splitter=splitter)
        nodes = await list_index.abuild(doc)
    encoder = EmbeddingModel.default()
    cross_encoder = CrossEncoder.default()
    relational_db = get_sessionmaker()
    vectordb_wiki = InMemoryVectorDB(encoder=encoder, cross_encoder=cross_encoder)
    with Timer("Adding embeddings to vectordb"):
        vectordb_wiki.add_all(nodes)

with Timer("Example dialogues"):
    dialogues = load_dialogues()
    vectordb_dialogue = InMemoryVectorDB(encoder=encoder, cross_encoder=cross_encoder)
    with Timer("Embedding dialogues"):
        for d in dialogues:
            d.embedding = encoder.encode_passage(d.content)
            d.embedding_model = encoder.name
    with Timer("Adding dialogues to vectordb"):
        vectordb_dialogue.add_all(dialogues)

2023-07-05 22:03:10.756 | INFO     | clonr.data.parsers.web:extract:17 - Fetching from url: https://chainsaw-man.fandom.com/wiki/Makima


Innate Characteristics
	Character name
	Short description
	Long description
		Load tokenizer
		Load splitter
		Pull data from web


2023-07-05 22:03:11.037 | INFO     | clonr.index:abuild:105 - Creating leaf nodes


		Chunk document
		Generating long description via 30 LLM calls
Wiki/document upload
	Creating index
	Adding embeddings to vectordb
Example dialogues
	Embedding dialogues
	Adding dialogues to vectordb


In [146]:
# unfiltered_content.content[-9755:]

## TODO
1. Memory formation. I think we just do something like every other message, record a memory. This is viewed as an internal thought/observation
2. Entity Context. We need to have the Clone periodically summarize what it knows and thinks about the user it is talking to
3. Agent summary. This is a running summarization of the Clone's thoughts, feelings, actions.
4. Previous conversation retrieval. Lots of pitfalls here if you retrieve without any context.
5. Generative-agents time-importance-similarity weighted retrieval
6. [optional] Reaction decision making. Whether to respond to a text or not
7. Output parsing text messages. How do we parse multi-line responses? separate on newline? make sure that's reflected in example dialogues
8. Reflection creation and retrieval.
9. Fact retrieval from the above wiki stuff
10. Token counting. Make sure all of this shit is within the context window and price it accordingly! which is about $0.6 cents per message. so 1k messages per $6.